# comma package tutorial

In [1]:
from comma.model import Model
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

pd.set_option('display.max_columns', None)
np.random.seed(0)
dir_params = '../parameters/'

### Step 1. Initialise the model


In [81]:
model = Model()

### Step 2. Generate n individuals by sampling from the IPF weights (previously computed in R)

Note that if you wish to generate a population based on parameters specified in `params_individual.json` simply set `use_ipf` to `False`. This will initiate a sampling from the distribution outlined in the json file, which does *not* take into account the joint probabilities within variables. Only the `use_ipf` set to `True` will sample with replacement from a distribution whereby the joint probabilities between variables is accounted for. 

In [82]:
size = 500
model.setup(size, '../parameters/', use_ipf=True) 

Populating individuals: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 26970.24i/s]


To see who are the agents, you can print their features:

### Step 3. Run the model

In [83]:
steps = 200

n = steps // 4
lockdown_pattern = ["absent"]*n + ["easy"]*n + ["hard"]*n + ["medium"]*n


[0, 50, 100, 150]

In [84]:
model.run(steps=steps, lockdown_policy=lockdown_pattern, out_path="simulation_results.csv")

Running simulation: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:20<00:00,  9.74it/s]


In [85]:
len(lockdown_pattern)

200

The model at the end of the run will output "simulation_results.csv". 

### Step 4. Plot the results

In [86]:
data = pd.read_csv('simulation_results.csv', delimiter=";", decimal=",")

In [87]:
display(data.head())

,step_id,lockdown,agent_id,delta_mental_health,cumulative_mental_health
0,0,absent,0,0.0,-4.52
1,0,absent,1,0.0,0.00
2,0,absent,2,0.0,0.00
3,0,absent,3,0.0,-3.94
4,0,absent,4,0.0,-1.43


In [88]:
data['lockdown'].unique()

array(['absent', 'easy', 'hard', 'medium'], dtype=object)

#### Group by step_id and compute the average mental health score

In [89]:
grouped_data = data.groupby(['step_id', 'lockdown']).mean().reset_index()
grouped_data

,step_id,lockdown,agent_id,delta_mental_health,cumulative_mental_health
0,0,absent,249.5,0.00000,-2.825900
1,1,absent,249.5,-2.51286,-5.340766
2,2,absent,249.5,-2.80204,-8.144796
3,3,absent,249.5,-2.48394,-10.630735
4,4,absent,249.5,-2.65358,-13.286333
...,...,...,...,...,...
195,195,medium,249.5,-22.18128,-2862.640247
196,196,medium,249.5,-21.93858,-2884.580814
197,197,medium,249.5,-22.66870,-2907.251515
198,198,medium,249.5,-22.26396,-2929.517436


### Cumulative mental health across time per agent

In [90]:
agg_df = data.groupby(['step_id', 'lockdown'])['cumulative_mental_health'].mean().reset_index()
agg_df['label'] = 'average mental health'

few_points = data[data['agent_id'] <= 3]
other_points = data[data['agent_id'] > 3]

max_step_ids = data.groupby('lockdown')['step_id'].max().reset_index()


In [91]:
# Create a dictionary with a unique color for each agent_id
colors = {}
unique_agents = data['agent_id'].unique()
for agent in unique_agents:
    if agent <= 3:
        colors[agent] = {0: "#386cb0", 1: "#fdb462", 2: "#7fc97f", 3: "darkred"}[agent]
    else:
        colors[agent] = "gainsboro"


In [94]:
fig, ax = plt.subplots(figsize=(10, 6))

# other_points
sns.lineplot(data=other_points, x='step_id', y='cumulative_mental_health', hue='agent_id', palette=colors, legend=False)
sns.scatterplot(data=other_points, x='step_id', y='cumulative_mental_health', hue='agent_id', palette=colors, s=30, legend=False)

# few_points
sns.lineplot(data=few_points, x='step_id', y='cumulative_mental_health', hue='agent_id', palette=colors, ax=ax)
sns.scatterplot(data=few_points, x='step_id', y='cumulative_mental_health', hue='agent_id', palette=colors, s=30, ax=ax)

# agg_df
sns.lineplot(data=agg_df, x='step_id', y='cumulative_mental_health', color='black', ax=ax, size=2)
sns.scatterplot(data=agg_df, x='step_id', y='cumulative_mental_health', color='black', s=20, ax=ax)

vector = [i * n for i in range(4)]

# add lockdown labels and dotted lines
for x_val, label in zip(vector, ['absent', 'easy lockdown', 'hard lockdown', 'medium lockdown']):
    ax.axvline(x=x_val, linestyle='--', color='gray')
    ax.text(x=x_val, y=ax.get_ylim()[1], s=label, va='bottom', ha='center')

avg_mh_text_y = agg_df[agg_df['step_id'] == agg_df['step_id'].max()]['cumulative_mental_health'].values[0]
ax.text(113, avg_mh_text_y, 'average mental health', va='center', ha='center', color='black')

ax.set_xlabel('step (1 step = 1 day)')
ax.set_ylabel('mental health')
ax.get_legend().remove()

plt.show()


### Average mental health for each step coloured by lockdown type

In [93]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=grouped_data, x='step_id', y='cumulative_mental_health', hue='lockdown')
plt.xlabel('Steps')
plt.ylabel('Average Mental Health')
plt.title('Mental health progression over time. 1 step = 1 day.')
plt.show()